# Cell Health Data Segmentor
### Find center coordinates for cells from Cell Health data

#### Import libraries

In [13]:
#from cellpose.io import logger_setup
from cellpose import models, core, io, utils

import pathlib
import pandas as pd

### Set Up CellPose

In [14]:
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)
#logger_setup();

>>> GPU activated? 0


In [15]:
import torch

torch.cuda.is_available()

False

### Helper functions for segmenting Cell Health data

In [ ]:
def segment_nuclei(DNA_image_path: pathlib.Path, save_path: pathlib.Path, cellpose_model: models.Cellpose):
    """finds center X,Y of nuclei and saves as tsv file

    Args:
        DNA_image_path (pathlib.Path): path to DNA channel image
        save_path (pathlib.Path): path to save nuclei locations tsv
        cellpose_model (models.Cellpose): cellpose model for segmenting nuclei
    """
    
    return None

segment_nuclei()